# End of Day Stock Research

Using only EOD price and volume data I provide a framework for how to go about developing a model that can be implemented as trading strategy.  The goal being to find a viable trading strategy that can run using EOD data and allows for entries at the closing price of the day.

Coming into this project I had some initial ideas for basic features to add.  I also am looking for a system with a holding period of 1-10 days that has a frequency of 4-20 trades per month.  These intial ideas combined with my trading experience forms the basis of my initial research as shown here.

### Getting Started - The Data

My goal is for this system to be a 1-10 day swing trading strategy.  I am therefore only interested in the end of day data for stocks.  For this project, I decided to use Quandl for my data.

WIKI EOD Stock Prices at https://www.quandl.com/data/WIKI-Wiki-EOD-Stock-Prices

### Load & Inspect Data

In [2]:
import pandas as pd
import numpy as np

In [1]:
#Loading the whole wiki prices file may take awhile, was 1.68GB at time of writing
#Also this part is not necessary, skip a few cells lower where we load just individual stocks
wiki_data = pd.read_csv('wiki_prices.csv')
wiki_data.shape

(14819949, 14)

In [2]:
wiki_data.head()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
0,A,1999-11-18,45.50,50.00,40.00,44.00,44739900.0,0.0,1.0,29.367758,32.272262,25.817810,28.399591,44739900.0
1,A,1999-11-19,42.94,43.00,39.81,40.38,10897100.0,0.0,1.0,27.714581,27.753307,25.694399,26.062292,10897100.0
2,A,1999-11-22,41.31,44.00,40.06,44.00,4705200.0,0.0,1.0,26.663343,28.399591,25.856536,28.399591,4705200.0
3,A,1999-11-23,42.50,43.63,40.25,40.25,4274400.0,0.0,1.0,27.436863,28.166360,25.984323,25.984323,4274400.0
4,A,1999-11-24,40.13,41.94,40.00,41.06,3464400.0,0.0,1.0,25.905464,27.073889,25.821544,26.505815,3464400.0


### Get Easier to Work With Data

A 1.68GB file that is basically a giant table index by stock ticker and then date is not going to be easy to work with.  To make this data a little easier to work with I wrote a script (process_quandl_data.py) that will process this big csv file and given a list of symbols, will write all data for that specific symbol to a new file 'symbol.csv'.

process_quandl_data.py available in this GitHub repo

In [4]:
import process_quandl_data as pqd

SYMBOLS = ['AAPL', 'TSLA', 'NFLX']
MAIN_DATA_PATH = 'wiki_prices.csv'

#Get symbol.csv files for every symbol in SYMBOLS
pqd.write_individual_stock_data_files(MAIN_DATA_PATH, SYMBOLS)

In [3]:
aapl = pd.read_csv('data/AAPL.csv')
nflx = pd.read_csv('data/NFLX.csv')
tsla = pd.read_csv('data/TSLA.csv')

In [6]:
aapl.shape

(9173, 14)

In [7]:
aapl.head()

,date,ticker,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
0,1980-12-12,AAPL,28.75,28.87,28.75,28.75,2093900.0,0.0,1.0,0.426158,0.427937,0.426158,0.426158,117258400.0
1,1980-12-15,AAPL,27.38,27.38,27.25,27.25,785200.0,0.0,1.0,0.405851,0.405851,0.403924,0.403924,43971200.0
2,1980-12-16,AAPL,25.37,25.37,25.25,25.25,472000.0,0.0,1.0,0.376057,0.376057,0.374278,0.374278,26432000.0
3,1980-12-17,AAPL,25.87,26.00,25.87,25.87,385900.0,0.0,1.0,0.383468,0.385395,0.383468,0.383468,21610400.0
4,1980-12-18,AAPL,26.63,26.75,26.63,26.63,327900.0,0.0,1.0,0.394733,0.396512,0.394733,0.394733,18362400.0


### Clean Up Data

Now time to remove any unnecssary columns, set the date field as the index column, and get our data ready to engineer some features.  Here I wrote a module 'get_data.py', call the get_data function with the argument 'symbol' at least and it will return a cleaned up data frame.

get_data.py is available in the GitHub repo

In [6]:
from get_data import get_data

aapl = get_data('aapl', with_features=False)
aapl.head()

,open,high,low,close,volume
date,,,,,
1980-12-12,0.426158,0.427937,0.426158,0.426158,2093900.0
1980-12-15,0.405851,0.405851,0.403924,0.403924,785200.0
1980-12-16,0.376057,0.376057,0.374278,0.374278,472000.0
1980-12-17,0.383468,0.385395,0.383468,0.383468,385900.0
1980-12-18,0.394733,0.396512,0.394733,0.394733,327900.0


So now as you can see the date is set as the index column.  The Adj open, high, low, close has replaced the nominal OHLC as this is what we will use.  Also dropped the columns not needed like split ratio and dividend.

### Add Features

The get_data module is where I engineer my features.

This part of the project is where you can spend as much time as you want.  The feature engineering is really the core of getting good results with an ML algorithm.  For this initial attempt and for this write up I will highlight a few of the features I began with.

Gap - % from yesterdays close to today's open  
Day Return from Open - % move from today's open to today's close  
Close as % of Bar - 1.0 meaning we closed at highs, 0.0 meaning closed a lows (0.5 close in middle of high/low range)  
SMA - Simple Moving Average  
ATR - Average True Range

In [7]:
#Get data, this time with features
aapl = get_data('aapl', start_date='2010-01-01', with_features=True)

In [8]:
aapl.head()

,open,high,low,close,volume,daily_ret,day_ret_from_open,gap,abs_gap,ATR1,ATRX,current_bar_bigger_than_avg,sma,close_above_sma,in_bar_range_1,in_bar_range_5,in_bar_range_20
date,,,,,,,,,,,,,,,,,
2010-01-04,27.652721,27.791354,27.516679,27.727867,17633200.0,1.555530,0.271752,1.280299,1.280299,0.274674,0.578183,0,25.870322,1,0.768868,0.920840,0.981022
2010-01-05,27.804310,27.932578,27.629399,27.775806,21496600.0,0.172889,-0.102516,0.275688,0.275688,0.303178,0.531152,0,25.894110,1,0.482906,0.833791,0.955035
2010-01-06,27.775806,27.885935,27.305491,27.333995,19720000.0,-1.590633,-1.590633,0.000000,0.000000,0.580444,0.527200,1,25.912327,1,0.049107,0.365385,0.828189
2010-01-07,27.435054,27.467445,27.085233,27.283465,17040400.0,-0.184860,-0.552538,0.369721,0.369721,0.382212,0.522665,0,25.933316,1,0.518644,0.233945,0.801820
2010-01-08,27.247187,27.467445,27.086529,27.464854,15986100.0,0.664830,0.798859,-0.132966,0.132966,0.380916,0.488007,0,25.971174,1,0.993197,0.448012,0.853192


It is a good idea to normalize the features relative to the stock and it's characteristics.  For example a raw price will not tell us much but a relative range (like the ATR) will be able to show if the stock has moved more than average and that could mean something.

Try to create features that are can be normalized in this manner, and therefore can be applied and compared across other stocks too.  Things like ratio to avg true range, ratio to avg volume, price / SMA, etc.

### Build ML Model

Time to build our first ML model.  This module is build_ml_model.py.  Now that we have our features we need to do a few things before we start training and testing a model.

First need to decide and add the label column - what are we predicting?

For this first run through here I decided I wanted to try to predict the next day's return.  (add_label_cols_to_df())

After that we need to split our data into train and test sets.  Even though we will use cross-validation, its good to withhold some of the data in a test set for a final evaluation.

After that's done, we can train and test a model.

In [9]:
import build_ml_model as bml

#Add label columns to dataset
aapl = bml.add_label_cols_to_df(aapl)
#Notice two new columns - 'next_day_return' and 'next_day_direction'
aapl.head()

,open,high,low,close,volume,daily_ret,day_ret_from_open,gap,abs_gap,ATR1,ATRX,current_bar_bigger_than_avg,sma,close_above_sma,in_bar_range_1,in_bar_range_5,in_bar_range_20,next_day_return,next_day_direction
date,,,,,,,,,,,,,,,,,,,
2010-01-04,27.652721,27.791354,27.516679,27.727867,17633200.0,1.555530,0.271752,1.280299,1.280299,0.274674,0.578183,0,25.870322,1,0.768868,0.920840,0.981022,0.172889,0
2010-01-05,27.804310,27.932578,27.629399,27.775806,21496600.0,0.172889,-0.102516,0.275688,0.275688,0.303178,0.531152,0,25.894110,1,0.482906,0.833791,0.955035,-1.590633,-1
2010-01-06,27.775806,27.885935,27.305491,27.333995,19720000.0,-1.590633,-1.590633,0.000000,0.000000,0.580444,0.527200,1,25.912327,1,0.049107,0.365385,0.828189,-0.184860,0
2010-01-07,27.435054,27.467445,27.085233,27.283465,17040400.0,-0.184860,-0.552538,0.369721,0.369721,0.382212,0.522665,0,25.933316,1,0.518644,0.233945,0.801820,0.664830,0
2010-01-08,27.247187,27.467445,27.086529,27.464854,15986100.0,0.664830,0.798859,-0.132966,0.132966,0.380916,0.488007,0,25.971174,1,0.993197,0.448012,0.853192,-0.882159,-1


In [11]:
#Choose which columns for features and label
FEATURE_LIST = ['day_ret_from_open', 'gap', 'abs_gap', 'ATR1', 'ATRX',
                'current_bar_bigger_than_avg', 'close_above_sma', 
                'in_bar_range_1', 'in_bar_range_5', 'in_bar_range_20']

LABEL_COL = ['next_day_direction']

In [14]:
#Now split the data
X_train, X_test, Y_train, Y_test, X_backtest = bml.get_train_and_test_sets(aapl, feature_cols=FEATURE_LIST, label_col=LABEL_COL)

### Run Cross-Validation

To avoid overfitting we can play around with our ML model on the training set and still get an accurate idea of how it is doing by using cross-validation.  It is important that we used a time series cross validation here.

In [15]:
from sklearn.ensemble import RandomForestClassifier

#Initiate ML learner
clf = RandomForestClassifier(n_estimators=100, min_samples_leaf=50)
bml.run_time_series_cross_validation(clf, X_train, Y_train)

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\model_selection\_validation.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


Avg. Score: 0.397196261682
Min Score: 0.331775700935
Max Score: 0.46261682243


array([ 0.42523364,  0.35046729,  0.3317757 ,  0.41588785,  0.46261682])

Not bad, average almost 40% accuracy in predicting next day direction.  It should be noted that next day direction as I coded into build_ml_model.py is one of three states:

Go Long == 1 == Next Day Return > 0.75%   
Go Short == -1 == Next Day Return < (0.75%)  
Stay Flat == 0 === Next Day Return between (0.75%) and 0.75%

So a 40% accuracy in predicting whether to stay flat, go long, or go short is a pretty good start with these as our classes.

### Train and Test ML Model

Now let's actual build a model to test on our out of sample data.

In [36]:
from sklearn.metrics import accuracy_score

#Instantiate model
clf = RandomForestClassifier(n_estimators=100, min_samples_leaf=50)
#Train model
clf.fit(X_train, Y_train)
#Get predictions - aka test model
predictions = clf.predict(X_test)
score = accuracy_score(Y_test, predictions)
print "Model Accuracy: " + str(score)

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Model Accuracy: 0.481012658228


So our out of sample performed better in this test with 48% accuracy.  Now this is a red flag as generally speaking out of sample shouldn't be performing better.  One thing to note is that n_estimators=100 is kind of low.  This can introduce all sorts of randomness in this RandomForest and not really give us a clear answer.  I kept it at 100 for the purpose of this notebook to keep things running faster.  This model should probably have between 1000-10000 trees in the random forest.

### Backtest

Now to get an idea of how well this model would hypothetically perform had we traded it at the closing price given the signal for tomorrow I quickly wrote backtester.py.  This assumes we have 0 slippage and commission, and that we can get the day's closing price.  This is a bit of a look ahead bias but there are order types which allow for getting the closing price (Market On Close), however the day is still trading so technically we'd be estimating all the features with 1 minute to go in the day and have 30 seconds to get our MOC orders in (I believe some MOC types can be sent up to 3:59:30pm).  So it is kind of possible to trade this on the close but there are a lot of assumptions going in our favor.  So this backtest.py is very, very optimistic and all sorts of costs need to be added to this backtest model to make it more realistic.

In [39]:
import backtester as bt

#Add predictions to backtest df
X_backtest.loc[:,'predictions'] = predictions

In [41]:
bt.get_total_return(X_backtest)

128
553
 0    425
 1     82
-1     46
Name: predictions, dtype: int64
TotalReturn:3.716688  #Trades:128  ReturnPerTrade:0.029037 


backtester.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['trade_return'] = df['predictions'] * df['next_day_return']


### Conclusion

Our simple model here predicted 128 different trades since 2010.  These results will fluctuate wildly with each 100 tree random forest as there is a lot of variance with so few estimators.  I intend to add more features, play around with different labels, and try different ML models.  However, this project illustrates the overall process and work flow for building an ML model to predict trading results

### Next Steps

Here are some future ideas to explore:

##### New Features
-Volume vs Avg Volume as bool, as ratio  
-Close/SMA ratio - indicate magnitude from SMA  
-Earnings Calendar - bool cool for is first day after earnings  

##### Different Labels
-Predict regression for next day's return, next 2-10 day's return  
-Predict class probabilities and raise threshold of minimum probability for entering a trade  

##### Other ML Approaches
-Regression prediction instead of classification  
-Other algorithms - XGB, SVM, NN  
-Build an ensemble of various learners  